In [4]:
from helper_modified import  graph_generator
import torch
from wl_test_l2O import wl_test
from wl_test_final import WLtest
torch.set_printoptions(profile="full")

In [5]:
info1 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/answer_eoe_gpt-4o-2024-08-06.lp')
info2 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/model.lp')
A1 = info1[0]
A2 = info2[0]
f1 = info1[1]#variable
f2 = info2[1]
c1 = info1[2]#constrain
c2 = info2[2]
wltest = WLtest(A1, c1,f1,A2,c2,f2)
print(wltest.test())

UnboundLocalError: local variable 'ub' referenced before assignment

In [9]:
c1.shape, c2.shape

(torch.Size([26, 2]), torch.Size([36, 2]))